In [1]:
import folium
from folium import plugins
import pandas as pd
import json
import math

In [2]:
min_lat = 56.880525
max_lat = 57.073682
min_lon = 23.942295
max_lon = 24.255976

map_riga = folium.Map(location=[(min_lat + max_lat) / 2, (min_lon + max_lon) / 2], zoom_start=11)

In [3]:
geojson_file = 'districts.json'
df = pd.read_csv('district_avg_psqm.csv', encoding='utf-8')
df2 = pd.read_csv('monthly_avg_psqm.csv', encoding='utf-8')
# df2 = df2.groupby(df2['apkaime'])
# df2 = df2['price_psqm'].mean().sort_values()
df2

Unnamed: 0.1  Unnamed: 0      district                address room_count  \
0               0           2   Čiekurkalns              Rusova 30          1   
1               1           5         Teika           Brīvības 231          1   
2               2          17        centrs                 Veru 3          2   
3               3          23     Pļavnieki             Kaudzīšu 6          1   
4               4          25  Vecmīlgrāvis  Vecmīlgrāvja 1. l. 49          2   
..            ...         ...           ...                    ...        ...   
806           806        4982        centrs        Rūpniecības 34А          4   
807           807        5004        centrs                Čaka 91          1   
808           808        5011        centrs            Tallinas 33          1   
809           809        5013     Pļavnieki             Kupriču 1A          2   
810           810        5033        Imanta        Kurzemes pr. 72          1   

      area floors house_type   price   latitude  longitude  quadrant  \
0     36.0    3/5    M. ģim.   280.0  56.986648  24.174007    1369.0   
1     34.0    4/5       103.   290.0  56.976679  24.176255    1243.0   
2     38.0    1/5     Renov.   450.0  56.961845  24.101000    1039.0   
3     33.0    2/9       602.   220.0  56.938987  24.198513     744.0   
4     50.0    4/5   LT proj.   250.0  57.028859  24.107060    1859.0   
..     ...    ...        ...     ...        ...        ...       ...   
806  146.0    3/6      Jaun.  1200.0  56.965975  24.104628    1103.0   
807   10.0    3/4    P. kara   140.0  56.957181  24.137762     984.0   
808   31.0    1/5    P. kara   280.0  56.962705  24.138035    1047.0   
809   54.0    3/5      Jaun.   630.0  56.938434  24.224635     749.0   
810   20.0    5/5       602.   220.0  56.954642  24.034750     900.0   

     price_psqm               apkaime  
0      7.777778           Čiekurkalns  
1      8.529412                 Teika  
2     11.842105                Centrs  
3      6.666667             Pļavnieki  
4      5.000000          Vecmilgrāvis  
..          ...                   ...  
806    8.219178  Pētersala-Andrejsala  
807   14.000000                Centrs  
808    9.032258           Grīziņkalns  
809   11.666667             Pļavnieki  
810   11.000000                Imanta  

[811 rows x 14 columns]

In [4]:
with open(geojson_file, encoding='utf-8') as f:
    geojson_data = json.load(f)

In [5]:
geojson_data

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'gid': 59, 'apkaime': 'Rumbula'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[24.207479354035634, 56.898296289483774],
      [24.211146704304895, 56.89670453163311],
      [24.213277060647524, 56.89821016967172],
      [24.21659105407348, 56.90022579428325],
      [24.217320481491445, 56.90082194144661],
      [24.21753186119787, 56.901544090690756],
      [24.224709138399255, 56.89800796397556],
      [24.228047476521333, 56.896616479038535],
      [24.227986035858564, 56.89658246217932],
      [24.230516839346212, 56.895283373166556],
      [24.231596503798407, 56.89472942047657],
      [24.23456301565925, 56.893205351063344],
      [24.236325749645466, 56.89229925991029],
      [24.23629146564725, 56.892279273173195],
      [24.238353328036922, 56.8912382426632],
      [24.23872174852832, 56.89105221876954

In [6]:
folium.TileLayer('cartodbdark_matter').add_to(map_riga)

In [7]:
merged_data = []


for feature in geojson_data['features']:
    if 'geometry' in feature:
        apkaime = feature['properties']['apkaime']
        district_data = df[df['apkaime'] == apkaime]
#         print(district_data)
        if not district_data.empty:
            feature['properties']['price_psqm'] = district_data['price_psqm'].values[0]
            
        merged_data.append(feature)


# Remove features with missing geometries
merged_data = [feature for feature in merged_data if 'geometry' in feature]


geojson_layer = folium.GeoJson(
    geojson_data,
    name="choropleth",
    style_function=lambda feature: {
        'fillColor': 'YlOrRd',
        'fillOpacity': 0.7,
        'lineOpacity': 0.2,
    },
    tooltip=folium.GeoJsonTooltip(fields=['apkaime', 'price_psqm'], labels=True),
).add_to(map_riga)

folium.Choropleth(
    geo_data=geojson_data,
    name="choropleth2",
    data=df2,
    columns=["apkaime", "price_psqm"],
    key_on="feature.properties.apkaime",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Value",
).add_to(map_riga)


for feature in geojson_layer.data['features']:
    print(2)
    properties = feature['properties']
    apkaime = properties['apkaime']
    try:
        properties['price_psqm']
        price_psqm = properties['price_psqm']
    except:
        price_psqm = 0
    apkaime = properties['apkaime']
    coordinates = feature['geometry']['coordinates'][0]  # Extract the first set of coordinates
    
    # Calculate the centroid of the polygon
    centroid_lat = sum(coord[1] for coord in coordinates) / len(coordinates)
    centroid_lon = sum(coord[0] for coord in coordinates) / len(coordinates)
    
    # Create a marker for the area name
    folium.Marker(
        [centroid_lat, centroid_lon],
#           popup = apkaime,
        popup=f'{apkaime}: {int(round(price_psqm, 0))} EUR/M²',
        icon=folium.Icon(color='blue', icon='info-sign'),
    ).add_to(map_riga)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [8]:
map_riga

In [9]:
properties

{'gid': 116, 'apkaime': 'Atgāzene', 'price_psqm': 1284.403669724771}

In [11]:
map_riga.save('psqm_per_district.html')

In [ ]:
map_riga

In [ ]:
map_riga.save('kkk_district.html')

In [ ]:
map_riga